# VacationPy
---

## Maria's Code

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,280.11,87,16,0.45,AU,1697570457
1,1,prado,-17.3411,-39.2208,299.25,69,0,5.35,BR,1697570457
2,2,port-aux-francais,-49.3500,70.2167,275.32,74,95,8.36,TF,1697570457
3,3,whitehorse,60.7161,-135.0538,281.58,82,75,2.57,CA,1697570351
4,4,hithadhoo,-0.6000,73.0833,300.60,76,8,6.49,MV,1697570457


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map plot
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
# Temps in KELVIN. 300K = 80F; 290K= 62F
hi_temp_filter_s = city_data_df["Max Temp"]<300
low_temp_filter_s = city_data_df["Max Temp"]>295
hi_humidity_filter_s = city_data_df["Humidity"]<60
low_humidity_filter_s = city_data_df["Humidity"]>20
cloud_filter_s = city_data_df["Cloudiness"]<10
wind_filter_s = city_data_df["Wind Speed"]<5

all_filter_s = hi_temp_filter_s & low_temp_filter_s & cloud_filter_s & hi_humidity_filter_s & low_humidity_filter_s & wind_filter_s
all_filter_s.sum()

14

In [17]:
ideal_city_data_df = city_data_df.loc[all_filter_s].dropna()

# Display sample data
print(ideal_city_data_df.shape)
ideal_city_data_df.head()

(14, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,mapimi,25.8167,-103.8500,295.81,30,0,2.96,MX,1697570458
40,40,al bawiti,28.3492,28.8659,296.11,47,0,2.49,EG,1697570463
65,65,conway,35.0887,-92.4421,296.04,40,0,3.09,US,1697570439
80,80,newman,37.3138,-121.0208,299.39,48,0,3.58,US,1697570469
175,175,san patricio,28.0170,-97.5169,298.07,29,0,0.74,US,1697570482


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# THERE'S a more EFFICIENT (less repetitive) way to do this - KEEP rather than DROP!!!
#HOTEL_COLS = ["City", "Country", "Lat", "Lng", "Humidity"]
#hotel_df = ideal_city_data_df[HOTEL_COLS]


hotel_df = ideal_city_data_df.copy()
hotel_df = hotel_df.drop("Date", axis='columns')
hotel_df = hotel_df.drop("Wind Speed", axis='columns')
hotel_df = hotel_df.drop("Cloudiness", axis='columns')
hotel_df = hotel_df.drop("Max Temp", axis='columns')

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
7,7,mapimi,25.8167,-103.8500,30,MX,
40,40,al bawiti,28.3492,28.8659,47,EG,
65,65,conway,35.0887,-92.4421,40,US,
80,80,newman,37.3138,-121.0208,48,US,
175,175,san patricio,28.0170,-97.5169,29,US,
199,199,awjilah,29.1081,21.2869,37,LY,
346,346,grantsville,40.5999,-112.4644,47,US,
353,353,carahue,-38.7000,-73.1667,32,CL,
362,362,sheridan,44.8333,-106.9173,38,US,
365,365,al kharijah,25.4514,30.5464,35,EG,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
    "filter":" ",
    "limit":limit,
    "bias":" ",
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] =f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mapimi - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
conway - nearest hotel: Fairfield Inn & Suites by Marriott Conway
newman - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
grantsville - nearest hotel: No hotel found
carahue - nearest hotel: No hotel found
sheridan - nearest hotel: Comfort Inn & Suites
al kharijah - nearest hotel: Kharga Hotel
brookhaven - nearest hotel: The Inn aon Whitworth
chihuahua - nearest hotel: No hotel found
harlingen - nearest hotel: Holiday Inn Express
white hall - nearest hotel: Days Inn & Suites by Wyndham Pine Bluff


,City_ID,City,Lat,Lng,Humidity,Country,Hotel Name
7,7,mapimi,25.8167,-103.8500,30,MX,No hotel found
40,40,al bawiti,28.3492,28.8659,47,EG,Old Oasis Hotel
65,65,conway,35.0887,-92.4421,40,US,Fairfield Inn & Suites by Marriott Conway
80,80,newman,37.3138,-121.0208,48,US,No hotel found
175,175,san patricio,28.0170,-97.5169,29,US,No hotel found
199,199,awjilah,29.1081,21.2869,37,LY,No hotel found
346,346,grantsville,40.5999,-112.4644,47,US,No hotel found
353,353,carahue,-38.7000,-73.1667,32,CL,No hotel found
362,362,sheridan,44.8333,-106.9173,38,US,Comfort Inn & Suites
365,365,al kharijah,25.4514,30.5464,35,EG,Kharga Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)

# Display the map plot
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)